# AOC 2022

Welcome to the Advent of Code 2022 !

## Basic configuration

In [ ]:
!pip install aocd

In [ ]:
import os

os.environ['AOC_SESSION'] = open('session.txt').read().strip()

from aocd import submit
from aocd.models import Puzzle

## Day 20
https://adventofcode.com/2022/day/20
### Part 1

In [ ]:
puzzle = Puzzle(2022, 20)
lines = puzzle.input_data.split('\n')

In [ ]:
values = list(map(int, lines))

In [ ]:
class Node:
    
    def __init__(self,v):
        self.v = v
        self.visited = False
        self.p = None
        self.n = None
    
def init_nodes(values, key=1):
    mix = [Node(v*key) for v in values]

    for n1, n2 in zip(mix[:-1], mix[1:]):
        n1.n = n2
        n2.p = n1

    mix[0].p = mix[-1]
    mix[-1].n = mix[0]
    
    return mix

def visit(node, head, N):

    v = node.v

    if v != 0:

        anchor = node

        if int((v/abs(v))) > 0:
            for i in range(abs(v)%(N-1)):
                anchor = anchor.n
        else:
            for i in range((abs(v) + 1 )%(N-1)):
                anchor = anchor.p

        if anchor != node:
            if node == head:
                head = node.n

            # detach node
            node.n.p = node.p
            node.p.n = node.n

            # attach node fwd
            node.n = anchor.n
            node.p = anchor

            # attach node bwd
            node.n.p = node
            node.p.n = node

    return head

def print_nodes(head):
    n = head
    print(n.v, end=", ")
    n = head.n
    while n != head:
        print(n.v, end=", ")
        n = n.n
        
def nodes_to_list(head):
    vals = []
    n = head
    vals.append(n.v)
    n = head.n
    while n != head:
        vals.append(n.v)
        n = n.n
    return vals

def grove_coords(head):
    vals = nodes_to_list(head)
    idx_0 = vals.index(0)
    return sum([vals[(idx_0+idx)%N] for idx in [1000, 2000, 3000]])

In [ ]:
mix = init_nodes(values)
N = len(mix)

head = mix[0]

for curr in mix:
    head = visit(curr, head, N)

In [ ]:
answer_a = grove_coords(head)

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
key = 811589153

mix = init_nodes(values, key)
N = len(mix)

for i in range(10):
    head = mix[0]
    
    for curr in mix:
        head = visit(curr, head, N)

In [ ]:
answer_b = grove_coords(head)

In [ ]:
puzzle.answer_b = answer_b

## Day 19
https://adventofcode.com/2022/day/19
### Part 1

In [ ]:
puzzle = Puzzle(2022, 19)
lines = puzzle.input_data.split('\n')

In [ ]:
import re
import numpy as np

r1 = re.compile(r".*costs (\d+) ore")
r2 = re.compile(r".*costs (\d+) ore and (\d+) .*")

def parse(s1, s2, s3, s4):
    costs = np.zeros((4, 4))

    costs[0, 0] = int(r1.match(s1).groups()[0])
    costs[1, 0] = int(r1.match(s2).groups()[0])
    
    g = r2.match(s3).groups()

    costs[2, 0] = g[0]
    costs[2, 1] = g[1]

    g = r2.match(s4).groups()

    costs[3, 0] = g[0]
    costs[3, 2] = g[1]

    return costs

blueprints = []

for line in lines:
    subs = line.split(':')[-1].split('.')[:4]
    blueprints.append(parse(*subs))

In [ ]:
def step(old_states, costs, max_t):
    
    states = []
    
    for prod, stock, t in old_states:
        # Construct
        for ri in range(4):
            if (stock < costs[ri,:]).any():
                continue
            if (ri != 3):
                if (prod[ri] >= costs[:,ri]).all():
                    continue
                if ((stock[ri] + prod[ri]*(max_t - t)) >= costs[:,ri]*(max_t - t)).all():
                    continue

            new_robots.append(ri)

        # Prod
        stock += prod

        for ri in new_robots:
            new_prod = prod.copy()
            new_prod[ri] += 1
            new_stock = stock - costs[ri,:]
            states.append((new_prod, new_stock, t+1))

        states.append((prod, stock, t+1))
    
    return states

def fitness(p,s,t, max_t):
    f = 0
    for i in range(4):
        f += p[i]*10**(2*i+1)
    f += (p[-1]*(max_t - t) + s[-1])*10**7
    return f

def run(costs, max_t):

    prod = np.array([1, 0, 0, 0])
    stock = np.zeros(4)
    states = [(prod, stock, 0)]

    for i in range(max_t):
        states = step(states, costs, max_t)
        states = sorted(states, key=lambda s: fitness(*s, max_t), reverse=True)[:50000]
    
    return max(s[1][-1] for s in states)

In [ ]:
level = 0
for idx, costs in enumerate(blueprints):
    level += (idx+1)*run(costs, 24)

In [ ]:
puzzle.answer_a = int(level)

### Part 2

In [ ]:
level = 1
for idx, costs in enumerate(blueprints[:3]):
    level *= run(costs, 32)

In [ ]:
puzzle.answer_b = int(level)

## Day 18
https://adventofcode.com/2022/day/18
### Part 1

In [ ]:
puzzle = Puzzle(2022, 18)
lines = puzzle.input_data.split('\n')

In [ ]:
cubes = [list(map(int, line.split(","))) for line in lines]

In [ ]:
def cube2faces(x,y,z):
    faces = []
    for dx in [0, 1]:
        faces.append((x+dx, y+0.5, z+0.5))
    for dy in [0, 1]:
        faces.append((x+0.5, y+dy, z+0.5))
    for dz in [0, 1]:
        faces.append((x+0.5, y+0.5, z+dz))
    return faces

def get_faces(cubes):
    faces = []

    for x, y, z in cubes:
        faces.extend(cube2faces(x, y, z))
        
    return faces

In [ ]:
from collections import Counter

faces = get_faces(cubes)

answer_a = sum(v for v in Counter(faces).values() if v < 2)

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
def diffuse(x, y, z, max_coord):
    pts = []
    for dx in [-1, 1]:
        pts.append((x+dx, y, z))
    for dy in [-1, 1]:
        pts.append((x, y+dy, z))
    for dz in [-1, 1]:
        pts.append((x, y, z+dz))
    return [pt for pt in pts if all(-1 <= coord <= max_coord+1 for coord in pt)]

def water_fill(cubes):
    
    maxc = max(max(coord for coord in cube) for cube in cubes)
    set_cubes = set([tuple(c) for c in cubes])

    water = set()
    steam = [(0, 0, 0)]

    cc = 0

    while steam:
        vapor = steam.pop(0)
        if vapor in water:
            continue
        steam.extend(set(diffuse(*vapor, maxc)) - set_cubes)

        water.add(vapor)
    
    return water

In [ ]:
water = water_fill(cubes)

water_faces = get_faces(water)
    
solo_faces = set(f for f, v in Counter(faces).items() if v < 2)

answer_b = len(set(water_faces) & solo_faces)

In [ ]:
puzzle.answer_b = answer_b

## Day 17
https://adventofcode.com/2022/day/17
### Part 1

In [ ]:
puzzle = Puzzle(2022, 17)
line = puzzle.input_data

In [ ]:
import numpy as np

pieces = [
    np.array([[1, 1, 1, 1]]).T, 
    np.array([[0, 1, 0], [1, 1, 1], [0, 1, 0]]), 
    np.array([[0, 0, 1], [0, 0, 1], [1, 1, 1]]),
    np.array([[1, 1, 1, 1]]),
    np.array([[1, 1], [1, 1]]),
]

pieces = [(idx+2)*p for idx, p in enumerate(pieces)] # for nicer viz

In [ ]:
def get_chamber(depth):

    chamber = np.zeros((9, depth))
    chamber[:,depth-1] = 1
    chamber[0,:] = 1
    chamber[8,:] = 1
    
    return chamber

def move(chamber, ax, ay, p, direction):
    nx, ny = ax, ay

    if direction == '>':
        nx += 1
    elif direction == '<':
        nx -= 1
    elif direction == 'v':
        ny += 1

    roi = chamber[nx:,ny:]
        
    if np.sum(roi[np.where(p)]) == 0:
        ax, ay = nx, ny
    return ax, ay

In [ ]:
def run(chamber, jets, nb_rocks, bottom_y):

    pcount = 0

    top = bottom_y - 4
    ax, ay = 3, top

    p = pieces[pcount]

    jidx = 0
    n_jets = len(jets)

    series = []

    while True:
        ax, ay = move(chamber, ax, ay, p, jets[jidx % n_jets])
        nx, ny = move(chamber, ax, ay, p, 'v')

        if (nx, ny) == (ax, ay):
            pcount += 1
            
            roi = chamber[nx:,ny:]
            roi[np.where(p)] = p[np.where(p)]

            top = min(top, ny)
            series.append((pcount, depth - top - 1))

            if pcount == nb_rocks:
                break

            p = pieces[pcount % len(pieces)]
            ay = top - 3 - p.shape[1]
            ax = 3
        else:
            ax, ay = nx, ny
        jidx += 1
        
    return series

In [ ]:
depth = 4000

chamber = get_chamber(depth)

series = run(chamber, line, 2022, depth - 1)

In [ ]:
delta = [b[1] - a[1] for a, b in zip(series, series[1:])]
answer_a = sum(delta[:-1])

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
depth = 20000

chamber = get_chamber(depth)

series = run(chamber, line, 10000, depth - 1)

In [ ]:
from scipy import fft, arange, signal

from matplotlib import pyplot as plt

delta = [b[1] - a[1] for a, b in zip(series, series[1:])]

sig = delta - np.mean(delta)

In [ ]:
# First take a look with Fourier

fft = np.fft.rfft(sig, norm="ortho")

def abs2(x):
    return x.real**2 + x.imag**2

# Compute auto-convolution
selfconv=np.fft.irfft(abs2(fft), norm="ortho")
selfconv=selfconv/selfconv[0]

plt.plot(selfconv)

In [ ]:
# Find the nice periodicity peaks

for freq in np.argsort(np.abs(selfconv))[::-1][1:6]:
    print(freq)

In [ ]:
# Confirm with autocorrelation

from statsmodels import api as sm

acf = sm.tsa.acf(sig, nlags=len(sig))

lag = arange(len(sig))
plt.plot(lag, acf)

In [ ]:
P = np.argsort(acf)[-2]

print(P) # confirmed

In [ ]:
# Use periodicity for direct computation

n = (1000000000000 // P) - 1
r = 1000000000000 % P

answer_b = sum(delta[:P]) + n*(sum(delta[P:2*P]))+sum(delta[P:P+r+1])

In [ ]:
puzzle.answer_b = answer_b

### Extra

In [ ]:
plt.rcParams["figure.figsize"] = (10, 10)

plt.imshow(chamber[:, -50:].T)

## Day 16
https://adventofcode.com/2022/day/16
### Part 1

In [ ]:
puzzle = Puzzle(2022, 16)
lines = puzzle.input_data.split('\n')

In [ ]:
import re

vertices = []

for line in lines:
    a, b = line.split(';')
    m = re.match(r"Valve (\w+) has flow rate=(\d+)", a)
    if m:
        src, rate = m.groups()
        rate = int(rate)
        m = re.match(r"(.*)valve(s?) (.*)", b)
        if m:
            dst = m.groups()[-1].split(', ')
        vertices.append((src, rate , dst))

In [ ]:
import networkx as nx

G = nx.Graph()

for src, rate, _ in vertices:
    G.add_node(src, rate=rate)

for src, _, dst in vertices:
    for tgt in dst:
        G.add_edge(src, tgt, weight=1)
        
apsp = dict(nx.all_pairs_shortest_path_length(G))
valves = {node: G.nodes[node]['rate'] for node in G.nodes if G.nodes[node]['rate'] > 0}

In [ ]:
from collections import defaultdict

def expand(path, timer, total_flow):
    paths = [(path, timer, total_flow)]
    visited = set(path)
    curr = path[-1]
    for node, rate in valves.items():
        if node not in visited:
            path_len = apsp[curr][node] + 1
            if timer > path_len:
                paths.append((path + [node], timer - path_len, total_flow+(timer - path_len)*rate))
    return paths

def reduce(paths):
    min_paths = defaultdict(list)

    new_paths = []
    
    for vertices, timer, flow in paths:
        min_paths[frozenset(vertices)].append((vertices, timer, flow))

    for sim_paths in min_paths.values():
        for v1, t1, f1 in sim_paths:
            if not any((f1 < f2) and (t1 < t2) for _, t2, f2 in sim_paths):
                new_paths.append((v1, t1, f1))
    return new_paths

paths = [(['AA'], 30, 0 )]

for i in range(10):
    new_paths = []

    for path in paths:
        new_paths.extend(expand(*path))
        
#     paths = reduce(new_paths) # not much faster
    paths = new_paths

In [ ]:
answer_a = sorted(paths, key=lambda t: t[2], reverse=True)[0][-1]

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
def expand_v2(p1, t1, f1, p2, t2, f2):
    paths = [(p1, t1, f1, p2, t2, f2)]
    visited = set(p1) | set(p2)
    c1 = p1[-1]
    c2 = p2[-1]
    for n1, r1 in valves.items():
        if n1 not in visited:
            p1_len = apsp[c1][n1] + 1
            if t1 > p1_len:
                for n2, r2 in valves.items():
                    if n2 not in visited and n1 != n2:
                        p2_len = apsp[c2][n2] + 1
                        if t2 > p2_len:
                            paths.append((p1 + [n1], t1 - p1_len, f1+(t1 - p1_len)*r1, p2 + [n2], t2 - p2_len, f2+(t2 - p2_len)*r2))
    return paths

In [ ]:
paths = [(['AA'], 26, 0, ['AA'], 26, 0 )]

for i in range(10):
    new_paths = []

    for path in paths:
        new_paths.extend(expand_v2(*path))

    paths = sorted(new_paths, key=lambda t: t[2]+t[5], reverse=True)[:1000000] # une heuristique c'est un algorithme en costume de clown :D

In [ ]:
best = sorted(new_paths, key=lambda t: t[2]+t[5], reverse=True)[0]
answer_b = best[2]+best[5]

In [ ]:
puzzle.answer_b = answer_b

## Day 15
https://adventofcode.com/2022/day/15
### Part 1

In [ ]:
puzzle = Puzzle(2022, 15)
lines = puzzle.input_data.split('\n')

In [ ]:
import re

class Sensor:
    
    def __init__(self, sx, sy, bx, by):
        self.pos = (sx, sy)
        self.beacon = (bx, by)
        self.radius = l1(sx, sy, bx, by)
        
    def cover(self):
        return self.pos[0]-self.radius, self.pos[1]-self.radius, self.pos[0]+self.radius, self.pos[1]+self.radius
    
    def corners(self):
        return [(self.pos[0]-self.radius, self.pos[1]), (self.pos[0], self.pos[1]-self.radius), (self.pos[0]+self.radius, self.pos[1]), (self.pos[0], self.pos[1]+self.radius)]

def parse(line):
    def match(sub):
        m = reg.match(sub)
        return list(map(int, m.groups()))

    s, b = line.split(':')
    return *match(s), *match(b)
    
def l1(x0, y0, x1, y1):
    return abs(x0-x1) + abs(y0 - y1)

reg = re.compile(r".*x=(-?\d+), y=(-?\d+)")

sensors = []

for line in lines:
    sx, sy, bx, by = parse(line)
    sensors.append(Sensor(*parse(line)))

In [ ]:
import numpy as np

yp = 2000000

xmin, xmax = min(s.cover()[0] for s in sensors), max(s.cover()[2] for s in sensors)

probe = np.zeros((xmax-xmin+1, 2))
probe[:,0] = range(xmin, xmax+1)
probe[:,1] = yp

sensing = np.array([False]*(xmax-xmin+1))

for s in sensors:
    sensing = np.logical_or(sensing, np.linalg.norm(probe - np.array([s.pos]*(xmax-xmin+1)), 1, axis=1) <= s.radius)
answer_a = len(np.where(sensing == True)[0]) - len(set([s.beacon for s in sensors if s.beacon[1] == yp]))

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
def line_intersection(line1, line2):
    # https://stackoverflow.com/questions/20677795/how-do-i-compute-the-intersection-point-of-two-lines
    xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])

    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]

    div = det(xdiff, ydiff)
    if div == 0:
        return None

    d = (det(*line1), det(*line2))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div
    return x, y

def non_integer(coord):
    ci = int(coord*10)
    return (ci % 5 == 0) and (ci % 10 != 0)

def grid_neighbors(x, y):
    dx = 0.5 if non_integer(x) else 1.0
    dy = 0.5 if non_integer(y) else 1.0
    return set([(int(x+xi), int(y+yi)) for xi in [-dx, 0, dx] for yi in [-dy, 0, dy]]) - {(x, y)}

def compute_intersections(sensors):
    intersections = set()

    for s0 in sensors:
        for s1 in sensors:
            c0 = s0.corners()
            c1 = s1.corners()
            c0.append(c0[0])
            c1.append(c1[0])
            for l0 in zip(c0[:-1], c0[1:]):
                for l1 in zip(c1[:-1], c1[1:]):
                    res = line_intersection(l0, l1)
                    if res is not None:
                        intersections.add(res)
    return intersections

def compute_neighbors(intersections, lx=4000000, ly=4000000):
    neighbors = set()
    for pt in intersections:
        neighbors.update(grid_neighbors(*pt))
        
    return [(x, y) for x, y in neighbors if (0 <= x <= lx) and (0 <= y <= ly)]
    
def probe_sensors_ext(neighbors):

    probe = np.array(neighbors)

    sensing = np.array([True]*len(neighbors))

    for s in sensors:
        sensing = np.logical_and(sensing, np.linalg.norm(probe - np.array([s.pos]*len(neighbors)), 1, axis=1) > s.radius)

    return np.array(neighbors)[np.where(sensing == True)[0]][0]

In [ ]:
intersections = compute_intersections(sensors)

neighbors = compute_neighbors(intersections)

bx, by = probe_sensors_ext(neighbors)

tuning_freq = bx*4000000 + by

In [ ]:
puzzle.answer_b = tuning_freq

## Day 14
https://adventofcode.com/2022/day/14
### Part 1

In [ ]:
puzzle = Puzzle(2022, 14)
lines = puzzle.input_data.split('\n')

In [ ]:
positions = set()

for line in lines:
    segment = list(map(eval, line.split(' -> ')))

    for p1, p2 in zip(segment[:-1], segment[1:]):
        if p1[0] == p2[0]:
            y0 = min(p1[1], p2[1])
            y1 = max(p1[1], p2[1])
            for y in range(y0, y1+1):
                positions.add((p1[0], y))
        else:
            x0 = min(p1[0], p2[0])
            x1 = max(p1[0], p2[0])
            for x in range(x0, x1+1):
                positions.add((x, p1[1]))


In [ ]:
mx, my = max(p[0] for p in positions), max(p[1] for p in positions)

In [ ]:
import numpy as np

cave = np.zeros((mx+1, my+1))
for pos in positions:
    
    cave[pos] = 2

In [ ]:
np.array(list(positions))

In [ ]:
def simulate_grain(cave, my):
    x, y = 500, 0
    
    while y < my:
        if cave[x, y+1] == 0:
            y += 1
        elif cave[x-1, y+1] == 0:
            x -= 1
            y += 1
        elif cave[x+1, y+1] == 0:
            x += 1
            y += 1
        else:
            return x, y

In [ ]:
for i in range(10000):
    res = simulate_grain(cave, my)
    if res is None:
        break
    cave[res[0], res[1]] = 1
answer_a = i

In [ ]:
puzzle.answer_a = answer_a

In [ ]:
from matplotlib import pyplot as plt

plt.rcParams["figure.figsize"] = (20,10)

plt.imshow(cave[250:550].T)

### Part 2

In [ ]:
cave = np.zeros((2*mx, my+3))
for pos in positions:
    cave[pos] = 2
    
cave[:,my+2] = 2

In [ ]:
for i in range(100000):
    res = simulate_grain(cave, my+3)
    cave[res[0], res[1]] = 1
    if res[0] == 500 and res[1] == 0:
        break
        
answer_b = i+1

In [ ]:
puzzle.answer_b = answer_b

In [ ]:
from matplotlib import pyplot as plt

plt.rcParams["figure.figsize"] = (20,10)

plt.imshow(cave[250:650].T)

## Day 13
https://adventofcode.com/2022/day/13
### Part 1

In [ ]:
puzzle = Puzzle(2022, 13)
lines = puzzle.input_data.split('\n\n')

In [ ]:
def recurse_comp(left, right):
    if type(left) != type(right):
        if type(left) is int:
            left = [left]
        else:
            right = [right]
    
    if type(left) == type(right) == int:
        if left != right:
            return left < right
    
    else:
        for l, r in zip(left, right):
            res = recurse_comp(l, r)
            if res is None:
                continue
            return res
        if len(left) != len(right):
            return len(left) < len(right)

In [ ]:
tot = 0

for idx, line in enumerate(lines):
    left, right = line.split()
    ret = recurse_comp(eval(left), eval(right))
    if ret:
        tot += idx + 1
        
answer_a = tot

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
messages = '\n'.join(lines).split('\n')
messages.extend(['[[2]]', '[[6]]'])
messages = list(map(eval, messages))

In [ ]:
from functools import cmp_to_key

def cmp_(a, b):
    ret = recurse_comp(a, b)
    if ret:
        return -1
    else:
        return 1
    return 0

messages.sort(key=cmp_to_key(cmp_))

In [ ]:
answer_b = (messages.index(eval('[[2]]')) + 1) * (messages.index(eval('[[6]]')) + 1)

In [ ]:
puzzle.answer_b = answer_b

## Day 12
https://adventofcode.com/2022/day/12
### Part 1

In [ ]:
puzzle = Puzzle(2022, 12)
lines = puzzle.input_data.split('\n')

In [ ]:
import numpy as np

for y, line in enumerate(lines):
    for x, c in enumerate(line):
        if c == 'S':
            start = (x, y)
        elif c == 'E':
            end = (x, y)

hmap = np.array([[ord(c) - ord('a') for c in line] for line in lines]).T
hmap[start] = 0
hmap[end] = ord('z') - ord('a')

### Part 1 (new, networkx)

In [ ]:
# Fucking networkx version
!pip install networkx

In [ ]:
import networkx as nx

def neighbors(src, vertices):
    x, y = src
    candidates = set([(x+1, y), (x-1, y), (x, y+1), (x, y-1)])
    candidates = candidates & vertices
    return [v for v in candidates if hmap[v] <= hmap[src] + 1]

def reachable(start, neighbors, hmap):
    on_map = [n for n in neighbors if valid(n, hmap)]
    return 

def valid(pos, hmap):
    H, W = hmap.shape
    return (0 <= pos[0] < H) & (0 <= pos[1] < W)

def setup_graph(hmap):

    G = nx.DiGraph()

    nnx, nny = hmap.shape
    vertices = set([(x, y) for x in range(nnx) for y in range(nny)])

    G.add_node(v for v in vertices)

    for v in vertices:
        for u in neighbors(v, vertices):
            G.add_edge(v, u)
            
    return G

In [ ]:
G = setup_graph(hmap)
answer_a = nx.shortest_path_length(G, start, end)

### Part 1 (old, a la mano)

In [ ]:
class Vertex:
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.dist = 10**4
        self.prev = None
        self.visited = False
        
    def pos(self):
        return (self.x, self.y)
        
    def neighbors(self, hmap):
        candidates = [(self.x+1, self.y), (self.x-1, self.y), (self.x, self.y+1), (self.x, self.y-1)]
        return reachable(self.pos(), candidates, hmap)
    
class Queue:
    
    def __init__(self, values):
        self.list = list(values)
        
    def pop(self):
        return self.list.pop(0)
        
    def sort(self):
        self.list.sort(key=lambda v: v.dist)
        
    def empty(self):
        return len(self.list) == 0

def reachable(start, neighbors, hmap):
    on_map = [n for n in neighbors if valid(n, hmap)]
    return [v for v in on_map if hmap[v] <= hmap[start] + 1]

def valid(pos, hmap):
    H, W = hmap.shape
    return (0 <= pos[0] < H) & (0 <= pos[1] < W)
    
from collections import deque

def run_djikstra(start, hmap):

    nx, ny = hmap.shape
    verticies = [Vertex(x, y) for x in range(nx) for y in range(ny)]

    vmap = {(v.x, v.y): v for v in verticies}
    vmap[start].dist = 0

    queue = Queue(verticies)
    queue.sort()
    
    while not queue.empty():
        u = queue.pop()

        for v in [vmap[pos] for pos in u.neighbors(hmap)]:
            if not v.visited:
                dist = u.dist + 1
                if dist < v.dist:
                    v.dist = dist
                    v.prev = u
        u.visited = True
        queue.sort()
    
    return vmap

In [ ]:
vmap = run_djikstra(start, hmap)

In [ ]:
answer_a = vmap[end].dist

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
G = setup_graph(hmap)

In [ ]:
stsp = dict(nx.single_target_shortest_path_length(G, end))

low_points = [(xi, yi) for xi, yi in zip(*np.where(hmap == 0))]

answer_b = min(stsp[src] for src in low_points if src in stsp)

In [ ]:
puzzle.answer_b = answer_b

## Day 11
https://adventofcode.com/2022/day/11
### Part 1

In [ ]:
puzzle = Puzzle(2022, 11)
lines = puzzle.input_data.split('\n\n')

In [ ]:
class Monkey:
    
    def __init__(self, description):
        self.items = list(map(int, description[1].split(': ')[1].split(', ')))
        self.operation = description[2].split(' = ')[1]
        self.test_value = int(description[3].split()[-1])
        self.test_true = int(description[4].split()[-1])
        self.test_false = int(description[5].split()[-1])
        self.business_level = 0
        
    def inspect(self, mod, divide=True):
        for old in self.items:
            worry = eval(self.operation) // 3 if divide else eval(self.operation)
            worry = worry % mod
            target = self.test_true if worry % self.test_value == 0 else self.test_false
            monkeys[target].items.append(worry)

        self.business_level += len(self.items)
        self.items = []


def init_monkeys():
    monkeys = []
    for line in lines:
        monkeys.append(Monkey(line.split('\n')))
    return monkeys

In [ ]:
ppmc = math.prod([monkey.test_value for monkey in monkeys])

monkeys = init_monkeys()

for i in range(20):
    for monkey in monkeys:
        monkey.inspect(mod=ppmc)

In [ ]:
import math

answer_a = math.prod(sorted([monkey.business_level for monkey in monkeys], reverse=True)[:2])

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
ppmc = math.prod([monkey.test_value for monkey in monkeys])

monkeys = init_monkeys()

for i in range(10000):
    for monkey in monkeys:
        monkey.inspect(mod=ppmc, divide=False)

In [ ]:
answer_b = math.prod(sorted([monkey.business_level for monkey in monkeys], reverse=True)[:2])

In [ ]:
puzzle.answer_b = answer_b

## Day 10
https://adventofcode.com/2022/day/10
### Part 1

In [ ]:
puzzle = Puzzle(2022, 10)
lines = puzzle.input_data.split('\n')

In [ ]:
cycled = []
for line in lines:
    if line[0] == 'a':
        cycled.extend(['noop', line])
    else:
        cycled.append(line)

In [ ]:
X = 1

values = [X]

for ins in cycled:
    if ins[0] == 'a':
        X += int(ins.split()[1])
    values.append(X)

In [ ]:
answer_a = sum([cycle*values[cycle-1] for cycle in range(20, 220+1, 40)])

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
import numpy as np

m = 40

screen = np.zeros((6, m))

for pos, val in enumerate(values[:-1]):
    l = pos // m
    r = pos % m
    screen[l, r] = 1 if abs(val - r) < 2 else 0

In [ ]:
from matplotlib import pyplot as plt

plt.rcParams["figure.figsize"] = (20,10)

plt.imshow(screen)

In [ ]:
answer_b = "RZEKEFHA"

In [ ]:
puzzle.answer_b = answer_b

## Day 9
https://adventofcode.com/2022/day/9
### Part 1

In [ ]:
puzzle = Puzzle(2022, 9)
lines = puzzle.input_data.split('\n')

In [ ]:
directions = []

for line in lines:
    a, b = line.split()
    directions.extend([a]*int(b))

In [ ]:
moves = {
    "D": (0, -1),
    "U": (0, 1),
    "L": (-1, 0),
    "R": (1, 0),
}

class Rope:
    
    def __init__(self):
        self.knots = []
        
    def add(self, knot):
        self.knots.append(knot)
        if len(self.knots) > 1:
            knot.link(self.knots[-2])
            
    def move(self, direction):
        self.knots[0].move(direction)
        for knot in self.knots[1:]:
            knot.follow()
            
    def head(self):
        return self.knots[0]
    
    def tail(self):
        return self.knots[-1]

class Knot:
    
    def __init__(self, x_=0, y_=0):
        self.x = x_
        self.y = y_
        self.next_ = None
        
    def pos(self):
        return self.x, self.y
        
    def link(self, other):
        self.next_ = other
        
    def move(self, direction):
        dx, dy = moves[direction]
        
        self.x += dx
        self.y += dy
        
    def follow(self):
        
        vx = self.next_.x - self.x
        vy = self.next_.y - self.y
        
        if max(abs(vx), abs(vy)) > 1:
            if abs(vx) > 0:
                self.x += int(vx/abs(vx))
            if abs(vy) > 0:
                self.y += int((vy)/abs(vy))
                
def move_rope(rope, directions):

    positions = {rope.tail().pos()}

    for direction in directions:
        rope.move(direction)

        positions.add(rope.tail().pos())
        
    return positions

In [ ]:
rope = Rope()
for i in range(2):
    rope.add(Knot())

answer_a = len(move_rope(rope, directions))

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
rope = Rope()
for i in range(10):
    rope.add(Knot())
    
answer_b = len(move_rope(rope, directions))

In [ ]:
puzzle.answer_b = answer_b

### Extra

In [ ]:
import numpy as np
import cv2 as cv

resolution = (960, 540)

def draw_rope(rope, positions=None):
    img = np.zeros(resolution[::-1])
    
    img = cv.cvtColor(img.astype('uint8'), cv.COLOR_GRAY2BGR)
    
    mx, my = -200, -350
    
    for idx, k in enumerate(rope.knots):
        img[k.x - mx, k.y - my,:] = (255 - 3*idx, 0, 255 - 3*idx)
        
    if positions is not None:
        n = len(positions)
        
        for idx, (x, y) in enumerate(positions):
            img[x - mx, y - my,:] = (125, 0, int(255*idx / n))

    return img
    
def animate_rope(rope, directions):
    
    fourcc = cv.VideoWriter_fourcc(*'mp4v')
    out = cv.VideoWriter('day09.mp4', fourcc, 30.0, resolution)

    out.write(draw_rope(rope))

    positions = [rope.tail().pos()]
    
    for direction in directions:
        rope.move(direction)

        positions.append(rope.tail().pos())
        out.write(draw_rope(rope, positions))
    
    out.release()

In [ ]:
animate_rope(rope, directions)

## Day 8
https://adventofcode.com/2022/day/8
### Part 1

In [ ]:
puzzle = Puzzle(2022, 8)
lines = puzzle.input_data.split('\n')

In [ ]:
forest = np.array([list(map(int, line)) for line in lines])

In [ ]:
import numpy as np

def visibility(line):
    top = -1
    mask = []
    for item in line:
        mask.append(top)
        top = max(top, item)
    return line > np.array(mask)

In [ ]:
left = np.array([visibility(line) for line in forest])
right = np.array([visibility(line[::-1])[::-1] for line in forest])
top = np.array([visibility(line) for line in forest.T]).T
bottom = np.array([visibility(line[::-1])[::-1] for line in forest.T]).T

answer_a = np.sum(left | right | top | bottom)

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
def los(line):
    if len(line) < 2:
        return 0
    
    top = line[0]
    
    for cnt, item in enumerate(line[1:]):
        if item >= top:
            break
    return cnt + 1

def viewing_score(x, y):
    # left, right, top, bottom
    return los(forest[y,x::-1]) * los(forest[y,x:]) * los(forest[y::-1,x]) * los(forest[y:,x])

In [ ]:
H, W = forest.shape
answer_b = max(viewing_score(x, y) for x in range(W) for y in range(H))

In [ ]:
puzzle.answer_b = answer_b

### Extra

In [ ]:
from matplotlib import pyplot as plt

plt.rcParams["figure.figsize"] = (20,10)

fig, (ax1, ax2) = plt.subplots(1, 2)

ax1.imshow(forest, cmap='YlGn')
ax1.set_title("Forest")

ax2.imshow(left | right | top | bottom)
ax2.set_title("Visible trees")

## Day 7
https://adventofcode.com/2022/day/7
### Part 1

In [ ]:
puzzle = Puzzle(2022, 7)
lines = puzzle.input_data.split('\n')

In [ ]:
from collections import deque, defaultdict
import re
import os

class File:
    
    def __init__(self, name_, size_):
        self._name = name_
        self._size = size_
        
    def size(self):
        return int(self._size)

class Directory:
    
    def __init__(self):
        self.dirs = []
        self.files = []
        
    def size(self):
        return sum(file_.size() for file_ in self.files) + sum(dir_.size() for dir_ in self.dirs)
    

In [ ]:
def current_path():
    return os.path.join(*list(path))

path = deque()
inodes = defaultdict(Directory)

for line in lines:
    if line[0] == '$':
        cmd = line[2:]
        if cmd[:2] == 'cd':
            loc = cmd[3:]
            if loc == '..':
                path.pop()
            else:
                path.append(loc)
    else:
        cur_path = current_path()

        if line[:3] == 'dir':
            dir_path = os.path.join(cur_path, line[4:])
            inodes[cur_path].dirs.append(inodes[dir_path])
        else:
            inodes[cur_path].files.append(File(*line.split()[::-1]))

In [ ]:
answer_a = sum(dir_.size() for dir_ in inodes.values() if dir_.size() <= 100000)

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
total_space = 70000000
target_free_space = 30000000

min_space_to_free = target_free_space - (total_space - inodes['/'].size())

In [ ]:
answer_b = min(dir_.size() for dir_ in inodes.values() if dir_.size() >= min_space_to_free)

In [ ]:
puzzle.answer_b = answer_b

### Part 2

## Day 6
https://adventofcode.com/2022/day/6
### Part 1

In [ ]:
puzzle = Puzzle(2022, 6)
lines = puzzle.input_data.split('\n')

In [ ]:
line = lines[0]
for pos in range(4,len(line)):
    if len(set(line[pos-4:pos])) == 4:
        break

In [ ]:
puzzle.answer_a = pos

### Part 2

In [ ]:
line = lines[0]
for pos in range(14,len(line)):
    if len(set(line[pos-14:pos])) == 14:
        break

In [ ]:
puzzle.answer_b = pos

## Day 5
https://adventofcode.com/2022/day/5
### Part 1

In [ ]:
puzzle = Puzzle(2022, 5)
lines = puzzle.input_data.split('\n')

In [ ]:
from collections import deque, defaultdict
import re

def init_crates(lines):
    crates = defaultdict(deque)

    for offset, line in enumerate(lines):
        if line[1] == '1':
            break
        for idx, elem in enumerate(line[1::4]):
            if elem != ' ':
                crates[idx].appendleft(elem)
    
    instructions = []

    for line in lines[offset+2:]:
        m = re.match(r"move (\d+) from (\d+) to (\d+)", line)
        instructions.append(tuple(map(int, m.groups())))
    
    return crates, instructions

In [ ]:
crates, instructions = init_crates(lines)

for nb, src, dst in instructions:
    crates[dst-1].extend([crates[src-1].pop() for i in range(nb)])

In [ ]:
answer_a = ''.join([crates[idx][-1] for idx in range(9)])

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
crates, instructions = init_crates(lines)

for nb, src, dst in instructions:
    crates[dst-1].extend([crates[src-1].pop() for i in range(nb)][::-1])

In [ ]:
answer_b = ''.join([crates[idx][-1] for idx in range(9)])

In [ ]:
puzzle.answer_b = answer_b

## Day 4
https://adventofcode.com/2022/day/4
### Part 1

In [ ]:
puzzle = Puzzle(2022, 4)
lines = puzzle.input_data.split('\n')

In [ ]:
ranges = []
for l in lines:
    pair = []
    for r in l.split(','):
        a, b = map(int, r.split('-'))
        pair.append(set(range(a,b+1)))
    ranges.append(pair)

In [ ]:
answer_a = len([r1 for r1, r2 in ranges if (r1 <= r2) or (r2 <= r1)])

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
answer_b = len([r1 for r1, r2 in ranges if r1 & r2])

In [ ]:
puzzle.answer_b = answer_b

## Day 3
https://adventofcode.com/2022/day/3
### Part 1

In [ ]:
puzzle = Puzzle(2022, 3)
lines = puzzle.input_data.split('\n')

In [ ]:
from collections import Counter

sacks = [(Counter(l[:len(l)//2]), Counter(l[len(l)//2:])) for l in lines]

In [ ]:
def prio(c):
    if c.islower():
        return ord(c) - ord('a') + 1
    return ord(c) - ord('A') + 27

answer_a = sum(prio(list(s[0].keys() & s[1].keys())[0]) for s in sacks)

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
sacks = [Counter(l) for l in lines]

answer_b = sum([prio(list(sacks[i].keys() & sacks[i+1].keys() & sacks[i+2].keys())[0]) for i in range(0, len(sacks), 3)])

In [ ]:
puzzle.answer_b = answer_b

## Day 2
https://adventofcode.com/2022/day/2
### Part 1

In [ ]:
puzzle = Puzzle(2022, 2)
lines = puzzle.input_data.split('\n')

In [ ]:
score = 0
for line in lines:
    a, b = line.split()
    a, b = ord(a) - ord('A'), ord(b) - ord('X')
    if a == b:
        score += 3
    if a == (b+1)%3:
        score += 0
    if a == (b+2)%3:
        score += 6
    score += b+1

In [ ]:
puzzle.answer_a = score

### Part 2

In [ ]:
## Short version
score = 0
for line in lines:
    a, b = line.split()
    a, b = ord(a) - ord('A'), ord(b) - ord('X')
    score += 3*b + 1 + (a + (b+2) % 3) % 3

In [ ]:
## Long version
score = 0
for line in lines:
    a, b = line.split()
    a, b = ord(a) - ord('A'), ord(b) - ord('X')
    
    score += 3*(b)

    score +=1
    if b == 0:
        score += (a+2)%3
    elif b==1:
        score += a
    elif b==2:
        score += (a+1)%3

In [ ]:
puzzle.answer_b = score

## Day 1
https://adventofcode.com/2022/day/1
### Part 1

In [ ]:
puzzle = Puzzle(2022, 1)

In [ ]:
lines = puzzle.input_data.split('\n\n')

In [ ]:
answer_a = max([sum(int(e) for e in l.split('\n')) for l in lines])

In [ ]:
puzzle.answer_a = answer_a

### Part 2

In [ ]:
answer_b = sum(sorted([sum(int(e) for e in l.split('\n')) for l in lines], reverse=True)[:3])

In [ ]:
puzzle.answer_b = answer_b